In [ ]:
#import sys  
#sys.path.insert(0, '/home/brunovieira/backup/Tamires_Experiments/Scripts')
#import freesuferdata_to_dataframe as fd

In [1]:
# fazer script funcionar
# add BN_Atlas.annot
# group by por atlas escolhido

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from nilearn import surface


def collector(participants_list, path_example):    
    data = []
    participants_list_completed = []
    participants_list_incompleted = []
    t=0
    for i, participant in enumerate(participants_list):
        print(i)
        feature_processed = []
        for hemisphere in ['lh', 'rh']:
            atlasEcono = "/label/"+str(hemisphere)+".Eco.annot"
            atlasBN = "/label/"+str(hemisphere)+".BN_Atlas.annot" 
            atlasDF = "/label/"+str(hemisphere)+".aparc.DKTatlas.annot" # o atlas padrao do fastsurfer é lh.aparc.mapped.annot 
            area = "/surf/"+str(hemisphere)+".area"
            curv = "/surf/"+str(hemisphere)+".curv"
            sulc = "/surf/"+str(hemisphere)+".sulc"
            thickness = "/surf/"+str(hemisphere)+".thickness"
            features = [atlasEcono, atlasBN, atlasDF, area, curv, sulc, thickness]
            for feature in features:
                try:
                    feature_processed.append(surface.load_surf_data(freesurfer_data_folder + participant + feature))
                except:
                    participants_list_incompleted.append(participant)
                    pass      
        data.append([participant, feature_processed])

    participants_list_incompleted = np.unique(participants_list_incompleted)
    participants_list_comp = [[i, sub] for i, sub in enumerate(participants_list) if sub not in participants_list_incompleted]
    participants_list_completed_index = pd.DataFrame(participants_list_comp)[0]
    participants_list_completed = pd.DataFrame(participants_list_comp)[1]
    
    all_info = [data, participants_list_completed_index, participants_list_completed, participants_list_incompleted] 
    return all_info


def vertices_to_structures(data, participants_list_completed_index):    
    df_dict = {"participant":[],"hemisphere":[],"atlasEcono":[],"atlasBN":[],"atlasDF":[], "area":[],"curv":[],"sulc":[],"thickness":[]}
    participants_list_completed_index = list(participants_list_completed_index)
    for i,part in enumerate(data):
        print(i)
        if i in participants_list_completed_index:
            frame_lh = np.array(part[1][0:7])
            df_dict['atlasEcono'].extend(frame_lh[0])
            df_dict['atlasBN'].extend(frame_lh[1]) 
            df_dict['atlasDF'].extend(frame_lh[2])
            df_dict['area'].extend(frame_lh[3])
            df_dict['curv'].extend(frame_lh[4])
            df_dict['sulc'].extend(frame_lh[5])
            df_dict['thickness'].extend(frame_lh[6])
            df_dict['participant'].extend([part[0]] * len(frame_lh[0]))
            df_dict['hemisphere'].extend(['left'] * len(frame_lh[0]))
            frame_rh = np.array(part[1][7:14])
            df_dict['atlasEcono'].extend(frame_rh[0])
            df_dict['atlasBN'].extend(frame_lh[1])
            df_dict['atlasDF'].extend(frame_rh[2])
            df_dict['area'].extend(frame_rh[3])
            df_dict['curv'].extend(frame_rh[4])
            df_dict['sulc'].extend(frame_rh[5])
            df_dict['thickness'].extend(frame_rh[6])
            df_dict['participant'].extend([part[0]] * len(frame_rh[0]))
            df_dict['hemisphere'].extend(['right'] * len(frame_rh[0]))
    return pd.DataFrame(df_dict)    
    
    
class SuferData:
    
    def __init__(self,
                 freesurfer_data_folder = "/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/",
                 path_base_mri = "/home/brunovieira/Tamires_Experiments/Bases_de_Dados/",
                 path_example = "/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/A00008326/"
                    ):
    
        participants_list = os.listdir(freesurfer_data_folder)
        participants_list = np.setdiff1d(participants_list, ['fsaverage'])

        all_info = collector(participants_list, path_example)
        self.data = all_info[0] 
        self.participants_list_completed_index = all_info[1]
        self.participants_list_completed = all_info[2]
        self.participants_list_incompleted = all_info[3]

        
    def save_files(self):
        self.Data_Vertices = vertices_to_structures(self.data, self.participants_list_completed_index)
        self.Data_Vertices.to_parquet(self.path_base_mri + "MRI_Data_Vertices_" + str(len(self.Data_Vertices.participant.unique())) + ".parquet")

        
    def monitor(self):
        print('Completed: '+str(len(self.participants_list_completed)))
        print('Incompleted: '+str(len(self.participants_list_incompleted)))


In [34]:
# path
freesurfer_data_folder = "/home/brunovieira/backup/Tamires_Experiments/Data/Nki_data/"
path_base_mri = "/home/brunovieira/backup/Tamires_Experiments/Bases_de_Dados/"
path_example = "/home/brunovieira/backup/Tamires_Experiments/Data/Nki_data/A00008326/"

# participants
participants_list = os.listdir(freesurfer_data_folder)
participants_list = np.setdiff1d(participants_list, ['fsaverage'])

# data format
df_dict = {"participant":[],"hemisphere":[],"atlasEcono":[],"atlasBN":[],"atlasDKT":[], "area":[],"curv":[],"sulc":[],"thickness":[]}
df_dict_ = {"atlasEcono":[],"atlasBN":[],"atlasDKT":[], "area":[],"curv":[],"sulc":[],"thickness":[]}
df_atlasBN = {}
participants_list_incompleted = []

for i, participant in enumerate(participants_list):
    print(i)
    for hemisphere in ['lh', 'rh']:
        
        # paths
        atlasEcono = ["/label/"+str(hemisphere)+".Eco.annot", 'atlasEcono']
        atlasBN = ["/label/"+str(hemisphere)+".BN_Atlas.annot", 'atlasBN']
        atlasDKT = ["/label/"+str(hemisphere)+".aparc.DKTatlas.annot", 'atlasDKT']
        area = ["/surf/"+str(hemisphere)+".area", 'area']
        curv = ["/surf/"+str(hemisphere)+".curv", 'curv']
        sulc = ["/surf/"+str(hemisphere)+".sulc", 'sulc']
        thickness = ["/surf/"+str(hemisphere)+".thickness", 'thickness']
        features = [atlasEcono, atlasBN, atlasDKT, area, curv, sulc, thickness]
        
        # vertice data
        try:
            for [feature, f] in features:
                data = surface.load_surf_data(freesurfer_data_folder + participant + feature)
                df_dict[f].extend(data)
                df_dict_[f].extend(data)
        except:
            participants_list_incompleted.append(participant) 
        df_dict['participant'].extend([participant] * len(data))
        df_dict['hemisphere'].extend([hemisphere] * len(data))
        
        # strutural data
        df_atlasBN_ = pd.DataFrame(df_dict_).groupby(by='atlasBN').mean().drop(columns=['atlasEcono', 'atlasDKT'])
        df_atlasBN_['participant'] = [participant] * len(df_atlasBN_)
        df_atlasBN_['hemisphere'] = [hemisphere] * len(df_atlasBN_)
        df_atlasBN[participant+hemisphere] = df_atlasBN_
        df_dict_ = {"atlasEcono":[],"atlasBN":[],"atlasDKT":[], "area":[],"curv":[],"sulc":[],"thickness":[]}
                

df_vertices = pd.DataFrame(df_dict)   
df_strutures_BN = pd.concat(df_atlasBN).droplevel(level=0).reset_index()
df_strutures_BN

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 

In [2]:
freesurfer_data_folder = "/home/brunovieira/backup/Tamires_Experiments/Data/Nki_data/"
path_base_mri = "/home/brunovieira/backup/Tamires_Experiments/Bases_de_Dados/"
path_example = "/home/brunovieira/backup/Tamires_Experiments/Data/Nki_data/A00008326/"

sd = SuferData(freesurfer_data_folder = freesurfer_data_folder,
               path_base_mri = path_base_mri,
               path_example = path_example)
sd.monitor()
sd.participants_list_incompleted

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

array(['A00013809', 'A00028177', 'A00031411', 'A00031578', 'A00035869',
       'A00037767', 'A00038424', 'A00039218', 'A00040798', 'A00052307',
       'A00053203', 'A00054153', 'A00054578', 'A00055727', 'A00055765',
       'A00055907', 'A00055921', 'A00056022', 'A00056746', 'A00057809',
       'A00057863', 'A00058516', 'A00059733', 'A00059734', 'A00060493',
       'A00060675', 'A00062085', 'A00062349', 'A00063425', 'A00064418',
       'A00064597', 'A00065262', 'A00065820', 'A00066534', 'A00066573',
       'A00066581'], dtype='<U9')

In [5]:
sd.data[1]

['A00008399',
 [array([27, 27, 27, ...,  6,  6,  6], dtype=int32),
  array([193, 193, 203, ...,  19,  19,  19], dtype=int32),
  array([ 5,  5, 11, ..., 28, 28, 28], dtype=int32),
  array([0.18895893, 0.11855987, 0.312513  , ..., 0.39409602, 0.18697552,
         0.3216219 ], dtype=float32),
  array([-0.35283712, -0.3191213 , -0.27926186, ..., -0.15341985,
         -0.15857993,  0.01035583], dtype=float32),
  array([-10.108584 ,  -9.586247 ,  -6.5083003, ...,  -3.2706475,
          -3.4777584,  -3.0189595], dtype=float32),
  array([1.5528197, 1.6350813, 2.259167 , ..., 3.6824942, 3.7809894,
         3.304576 ], dtype=float32),
  array([28, 28, 28, ...,  6,  6,  6], dtype=int32),
  array([204, 204, 204, ...,  14,  14,  14], dtype=int32),
  array([11, 11, 11, ..., 28, 28, 28], dtype=int32),
  array([0.3500838 , 0.26483038, 0.30095097, ..., 0.63437635, 0.11868868,
         0.28751412], dtype=float32),
  array([-0.31286016, -0.28012195, -0.23669638, ..., -0.41191337,
         -0.6100871 , -0

In [ ]:
sd.Data_Vertices = vertices_to_structures(sd.data, sd.participants_list_completed_index)
sd.Data_Vertices